In [4]:
f

NameError: name 'f' is not defined

In [7]:
import numpy as np
#import dask
from pathlib import Path
from ViennaRNA import RNA
import pandas as pd
#from dask.distributed import Client
from multiprocessing import cpu_count
n_cores = cpu_count()
print(n_cores)
#from dask.distributed import progress
# client = Client()
# client

48


### client

In [8]:
class CFG:
    path = Path('../data/')

In [5]:


# # Function to perform RNA secondary structure prediction on a Pandas DataFrame
# def predict_structure(df):
#     df['ss_full'], df['ss_full_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x).mfe()))
#     return df



In [6]:


def read_and_filter_parquet(file_path):
    df = dd.read_parquet(file_path)
    df_filtered = df[['sequence_id', 'sequence']]
    df_filtered = df_filtered.drop_duplicates(subset=['sequence_id'])
    return df_filtered

def predict_structure(df):
    df['ss_full'], df['ss_full_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x).mfe()))
    df['ss_roi'], df['ss_roi_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x[26:-21]).mfe()))
    return df

# File path configuration (replace with your own path)
# Read and filter the DataFrame
dask_df_filtered = read_and_filter_parquet(CFG.path/'train_data.parquet')

# Repartition DataFrame to have as many partitions as cores
dask_df_filtered = dask_df_filtered.repartition(npartitions=32)

# Sample 100% of the rows (i.e., keep all rows)
sampled_dask_df = dask_df_filtered.sample(frac=1.0, random_state=42)

# Metadata for new columns
meta = sampled_dask_df._meta.assign(
    ss_full=pd.Series(dtype='str'), 
    ss_full_mfe=pd.Series(dtype='float64'),
    ss_roi=pd.Series(dtype='str'),
    ss_roi_mfe=pd.Series(dtype='float64')
)

# Apply RNA secondary structure prediction
predicted_dask_df = sampled_dask_df.map_partitions(predict_structure, meta=meta)

# Show progress (this will work if you are in a Jupyter Notebook)
progress(predicted_dask_df)

# Compute to get the result
predicted_df = predicted_dask_df.compute()

# Print the result
print(predicted_df)

# Close the Dask Client
client.close()
predicted_df.to_parquet('train_ss_vienna_rna.parquet', index=False)

ValueError: No clients found
Start a client and point it to the scheduler address
  from distributed import Client
  client = Client('ip-addr-of-scheduler:8786')


In [ ]:


def read_and_filter_parquet(file_path):
    df = dd.read_csv(file_path)
    df_filtered = df[['sequence_id', 'sequence']]
    df_filtered = df_filtered.drop_duplicates(subset=['sequence_id'])
    return df_filtered

def predict_structure(df):
    df['ss_full'], df['ss_full_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x).mfe()))
    df['ss_roi'], df['ss_roi_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x[26:-21]).mfe()))
    return df

# File path configuration (replace with your own path)
# Read and filter the DataFrame
dask_df_filtered = read_and_filter_parquet(CFG.path/'test_sequences.csv')

# Repartition DataFrame to have as many partitions as cores
dask_df_filtered = dask_df_filtered.repartition(npartitions=32)

# Sample 100% of the rows (i.e., keep all rows)
sampled_dask_df = dask_df_filtered

# Metadata for new columns
meta = sampled_dask_df._meta.assign(
    ss_full=pd.Series(dtype='str'), 
    ss_full_mfe=pd.Series(dtype='float64'),
    ss_roi=pd.Series(dtype='str'),
    ss_roi_mfe=pd.Series(dtype='float64')
)

# Apply RNA secondary structure prediction
predicted_dask_df = sampled_dask_df.map_partitions(predict_structure, meta=meta)

# Show progress (this will work if you are in a Jupyter Notebook)
progress(predicted_dask_df)

# Compute to get the result
predicted_df = predicted_dask_df.compute()

# Print the result
print(predicted_df)

# Close the Dask Client
client.close()
predicted_df.to_parquet('test_ss_vienna_rna.parquet', index=False)

In [ ]:
predicted_df

In [9]:
def predict_structure(df):
    df['ss_full'], df['ss_full_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x).mfe()))
    return df

df = pd.read_csv('../data/rmdb_data.v1.3.0.csv')[['sequence_id', 'sequence']].drop_duplicates(subset=['sequence_id'])
df = predict_structure(df)

In [11]:
df.to_parquet('rmdb_data.v1.3.0_ss.parquet', index=False)

In [10]:
def predict_structure(df):
    df['ss_full'], df['ss_full_mfe'] = zip(*df['sequence'].apply(lambda x: RNA.fold_compound(x).mfe()))
    return df

df = pd.read_csv('../data/R1138v1_m2.csv')[['sequence_id', 'sequence']]
df = predict_structure(df)
df.to_parquet('R1138v1_m2.csv.parquet', index=False)